In [ ]:
import pandas as pd 
import numpy as np 

from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm

In [ ]:
text_value = "작업전 안전교육 실시와 안전관리자 안전점검 실시를 통한 재발 방지 대책 및 향후 조치 계획"

# 인덱스 0부터 963까지 총 964행을 가진 DataFrame 생성
sub1 = pd.DataFrame({"sub": [text_value] * 964})

In [ ]:
sub2 = pd.read_csv("train2.csv")

In [ ]:
sub2 = sub2['templeate']

In [ ]:
sub3 = pd.read_csv("data/submissionV13.csv")
sub4 = pd.read_csv("data/submissionV18ckp3000.csv")

In [ ]:
sub3 = sub3['재발방지대책 및 향후조치계획']
sub4 = sub4['재발방지대책 및 향후조치계획']

In [ ]:
df = pd.concat([sub1, sub2, sub3, sub4], axis=1)


In [ ]:
df.columns = ['sub1', 'sub2', 'sub3', 'sub4']


In [ ]:
df.columns

In [ ]:
df = df[:964]

In [ ]:
from sentence_transformers import SentenceTransformer

# Embedding Vector 추출에 활용할 모델(jhgan/ko-sbert-sts) 불러오기
model = SentenceTransformer('jhgan/ko-sbert-sts', use_auth_token=False)

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

# 예시: df는 sub1, sub2, sub3, sub4 컬럼을 포함하는 데이터프레임
# model은 텍스트를 벡터로 인코딩하는 모델입니다.

# 각 행별 대표 텍스트와 유사도 정보를 저장할 리스트 초기화
representative_texts = []  # 각 행별 대표 텍스트 저장
cosine_scores = []         # 각 행별 대표 텍스트와 다른 텍스트 간 유사도 기록

# 모델을 통해 텍스트를 벡터로 인코딩하는 예시 함수가 있다고 가정합니다.
# 예: model.encode(text)
# model은 이미 초기화된 인코딩 모델이어야 합니다.

# 각 행에 대해 sub1~sub5 컬럼의 텍스트를 비교
for idx, row in tqdm(df.iterrows(), total=len(df)):
    # sub1부터 sub5까지의 텍스트를 리스트로 추출합니다.
    texts = [row[f"sub{i}"] for i in range(1, 5)]
    
    # 각 텍스트를 벡터화합니다.
    vectors = np.stack([model.encode(text) for text in texts])
    
    # 텍스트들 간의 코사인 유사도 행렬 계산
    similarity = cosine_similarity(vectors, vectors)
    
    # 각 텍스트에 대해 다른 텍스트들과의 평균 유사도를 계산합니다.
    avg_similarity = similarity.mean(axis=1)
    
    # 평균 유사도가 가장 높은 텍스트의 인덱스를 선택합니다.
    rep_idx = avg_similarity.argmax()
    
    # 대표 텍스트와 해당 텍스트의 유사도 벡터를 저장합니다.
    representative_texts.append(texts[rep_idx])
    cosine_scores.append(similarity[rep_idx].tolist())

# 결과를 DataFrame의 새로운 컬럼에 추가합니다.
df["대표재발방지대책"] = representative_texts
df["대표유사도"] = cosine_scores

# 결과 출력
print(df.head())

In [ ]:
df['대표재발방지대책']

In [ ]:


# 문장 리스트를 입력하여 임베딩 생성
pred_embeddings = model.encode(df['대표재발방지대책'])

In [ ]:
submission = pd.read_csv("sample_submission.csv", encoding = 'utf-8-sig')
# 최종 결과 저장
submission.iloc[:,1] = df['대표재발방지대책']
submission.iloc[:,2:] = pred_embeddings
submission.head()
# 최종 결과를 CSV로 저장
submission.to_csv('신기술V2.csv', index=False, encoding='utf-8-sig')